In [1]:
#r "nuget:Microsoft.ML"
#r "nuget:MathNet.Filtering"
#r "nuget:LINQtoCSV"

Installing package Microsoft.ML.........................................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package MathNet.Filtering...........done!

Successfully added reference to package MathNet.Filtering, version 0.6.0

Installing package LINQtoCSV..........done!

Successfully added reference to package LINQtoCSV, version 1.5.0.0

In [2]:
using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;
using LINQtoCSV;

In [3]:
public class Sample
{ 
    [CsvColumn(FieldIndex = 1)]
    public int Raw {get; set;}

    [CsvColumn(FieldIndex = 2)]
    public int Filtered {get; set;}
}
public class MetaItem
{
    [CsvColumn(FieldIndex = 1)]
    public int Ignore1 {get; set;}

    [CsvColumn(FieldIndex = 2)]
    public int Ignore2 {get; set;}
    
    [CsvColumn(FieldIndex = 3)]
    public string Name {get; set;}
    
    [CsvColumn(FieldIndex = 4)]
    public string Value {get; set;}
}

In [4]:
var dataPath = Path.Combine(Environment.CurrentDirectory, "psk_x20_20191221_1256.csv");

var inputFileDescription = new CsvFileDescription
{
    SeparatorChar = ',',
    FirstLineHasColumnNames = false, // no column names in first record
    EnforceCsvColumnAttribute = true,
    MaximumNbrExceptions = int.MaxValue,
    IgnoreUnknownColumns = true
};
var cc1 = new CsvContext();
var metaLines = cc1.Read<MetaItem>(dataPath, inputFileDescription);
var count = 0;
var samplingRate = int.Parse(metaLines
    .TakeWhile( r => { ++count; return r.Name != "BEGIN_DATA";})
    .Where( t => t.Name == "sampling_rate").Last().Value);
Console.WriteLine($"Sampling rate: {samplingRate}");
if (samplingRate != 400) throw new Exception("bad sampling rate");
var samples = cc1.Read<Sample>(dataPath, inputFileDescription).Skip(count+1).ToArray();
Console.WriteLine($"Loaded {samples.Count()} samples");

Sampling rate: 400
Loaded 28415 samples


In [24]:
var count = 0;

var rawData = new Graph.Scatter()
{
    x = samples.Select( s => (count ++) * 1.0d / samplingRate),
    y = samples.Select( s => s.Raw - 92000),
    mode = "lines",
    line = new Graph.Line()
};
var count2 = 0;
var filteredData = new Graph.Scatter()
{
    x = samples.Select( s => (count2 ++) * 1.0d / samplingRate),
    y = samples.Select( s => s.Filtered ),
    mode = "lines",
    line = new Graph.Line()
};

var chart = Chart.Plot(new [] {filteredData, rawData});
chart.Width = 1200;
chart.Height = 600;
display(chart);